In [36]:
# Importing libraries
import pandas as pd
import numpy as np
import cvxpy as cp

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [37]:
# Function to load files from csv file
def loadCVS(fileName):
    data = pd.read_csv(fileName, header = None)

    # Label values from 0 to -1
    data.loc[data[0] == 0, 0] = -1

    # Separate features and labels
    X = data.drop([0], axis = 1).to_numpy()
    Y = data[0].to_numpy()

    return X, Y

In [38]:
# Load CSV files into train and test
X_trainLoad, Y_trainLoad = loadCVS('train.csv')
X_test, Y_test = loadCVS('test.csv')

# Separate data into train and validation sets
X_train, X_validation = X_trainLoad[:4000, :], X_trainLoad[4000:, :]
Y_train, Y_validation = Y_trainLoad[:4000], Y_trainLoad[4000:]

In [39]:
# Training algorithm of soft-margin Linear SVM in primal form
def svm_train_primal(data_train, label_train, regularisation_para_C):
    numData, numFeatures = np.shape(data_train)

    # Variable setting
    weight = cp.Variable(shape=(numFeatures, 1), name='weight')
    bias = cp.Variable(name='bias')
    c = regularisation_para_C
    slack = cp.Variable(shape=(numData, 1), name='slack')

    # Solve
    findMin = cp.Minimize(0.5*cp.square(cp.norm(weight,2)) + c/numData * cp.sum(slack))
    findConstraints = [cp.multiply(label_train.reshape(-1,1), data_train @ (weight+bias)) >= 1 - slack, slack >= 0]
    answer = cp.Problem(findMin, findConstraints)
    answer.solve()

    return answer

In [40]:
# Testing algorithm of soft-margin Linear SVM in primal form
def svm_predict_primal(data_test, label_test, svm_model):
    # Variable setting
    weight = svm_model.var_dict['weight'].value
    bias = svm_model.var_dict['bias'].value

    # Solve
    Y = data_test @ weight + bias
    Y[Y > 0] = 1
    Y[Y <= 0] = -1
    answer = accuracy_score(label_test.reshape(-1, 1), Y)

    return answer

In [41]:
# Question 2 run
regularisation_para_C = 100
svm_model = svm_train_primal (X_train, Y_train, regularisation_para_C)
test_accuracy = svm_predict_primal (X_test, Y_test, svm_model)

# Solution of b 
print(f"\n Solution of b : {svm_model.var_dict['bias'].value}")
# Sum of all dimensions of w solution
print(f"\n Solution of w : {np.sum(svm_model.var_dict['weight'].value)}")
# Accuracy
print(f"\n Test Accuracy : {test_accuracy}")


 Solution of b : 0.0015240663360743323

 Solution of w : -2.0496487840304667e-10

 Test Accuracy : 0.956
